# Задача:
    
### *У нас появился запрос из отдела продаж и маркетинга.*
### Как вы знаете «МегаФон» предлагает обширный набор различных услуг своим абонентам.
### При этом разным пользователям интересны разные услуги.
### **Поэтому необходимо построить  алгоритм, который для каждой пары пользователь-услуга определит вероятность подключения услуги.**


## Данные:
*В качестве исходных данных вам будет доступна информация об отклике абонентов на предложение подключения одной из услуг. Каждому пользователю может быть сделано несколько предложений в разное время, каждое из которых он может или принять, или отклонить.*

*Отдельным набором данных будет являться нормализованный анонимизированный набор признаков, характеризующий профиль потребления абонента. Эти данные привязаны к определенному времени, поскольку профиль абонента может меняться с течением времени.*

*Данные ***train*** и ***test*** разбиты по периодам – на ***train доступно 4 месяцев***, а на ***test отложен последующий месяц***.* 



### Итого, в качестве входных данных будут представлены:
>* ### ***data_train.csv:***
    >>* id,
    >>* vas_id,
    >>* buy_time,
    >>* target
>* ### ***features.csv.zip:***
    >>* id,
    >>* \<feature_list\> 


### И тестовый набор:

>* ### ***data_test.csv:***
    >>* id,
    >>* vas_id,
    >>* buy_time
    
>* ### target - целевая переменная, где
    >>* 1 - означает подключение услуги,
    >>* 0 - абонент не подключил услугу соответственно. 
>* ### buy_time - время покупки,
    >>* представлено в формате timestamp, для работы с этим столбцом понадобится функция ***datetime.fromtimestamp*** из модуля ***datetime***.
>* ### id - идентификатор абонента
>* ### vas_id - подключаемая услуга

*Примечание: Размер файла features.csv в распакованном виде весит 20 гб, для работы  с ним можно воспользоваться pandas.read_csv, либо можно воспользоваться библиотекой Dask.*


## Метрика
*Скоринг будет осуществляться функцией ***f1***, невзвешенным образом,
как например делает функция 
***sklearn.metrics.f1_score(…, average=’macro’)***.*

[sklearn.metrics.f1_score — scikit-learn 0.22.1 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score)


## Формат представления результата
>1. Работающая модель в формате ***pickle***, которая принимает файл ***data_test.csv*** из корневой папки и записывает в эту же папку файл ***answers_test.csv***. В этом файле должны находится 4 столбца:
    >>* buy_time,
    >>* id,
    >>* vas_id
    >>* target.  
    >>>***Target можно записать как вероятность подключения услуги***.
>2.	Код модели можно представить в виде ***jupyter-ноутбука***. 
>3.	Презентация в формате ***.pdf***, в которой необходимо отразить:
>>* Информация о модели, ее параметрах, особенностях и основных результатах.
>>* Обоснование выбора модели и ее сравнение с альтернативами.
>>* Принцип составления индивидуальных предложений для выбранных абонентов.

*Рекомендуемое количество слайдов: ***5 – 10***.*

Файл ***answers_test.csv*** с результатами работы модели, презентацию, ноутбуки и резюме необходимо прикрепить ко второму уроку “курсовой проект”.


### Импорт библиотек

In [1]:
import pandas as pd
import dill
import dask.dataframe as dd
from datetime import datetime
from dask.distributed import Client, progress

### Запуск "клиента" Dask
#### *Client() предоствит панель мониторинга [Dashboard](http://127.0.0.1:8787/status), которая полезна для получения информации о вычислениях.*


In [2]:
%%time
client = Client()
client

CPU times: user 478 ms, sys: 89.3 ms, total: 567 ms
Wall time: 1.53 s


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 5.72 GiB
Comm: tcp://127.0.0.1:41367,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 5.72 GiB
Comm: tcp://127.0.0.1:41511,Total threads: 1
Dashboard: http://127.0.0.1:33939/status,Memory: 1.43 GiB
Nanny: tcp://127.0.0.1:38781,


In [3]:
PATH_D = './data/'

In [4]:
%%time
dd_features = dd.read_csv(f'{PATH_D}features.csv',sep='\t')
dd_train = dd.read_csv(f'{PATH_D}data_train.csv')
dd_test = dd.read_csv(f'{PATH_D}data_test.csv')

CPU times: user 132 ms, sys: 14.4 ms, total: 146 ms
Wall time: 149 ms


In [5]:
def print_shape_data(dd_data):
    print(f"Shape: columns: {len(dd_data.columns)} Items: {len(dd_data.index)}")
    
def get_replace_dict(col_ending):
    """Функция возврата словаря для замены columns['Unnamed: 0', 'buy_time'] 
    через функцию .rename(columns=dict(...))"""
    return {'Unnamed: 0': f'index_{col_ending}', 'buy_time': f'buy_time_{col_ending}'}

In [6]:
%%time
dd_train.head(3)

CPU times: user 88.2 ms, sys: 14.6 ms, total: 103 ms
Wall time: 1.41 s


,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0


In [7]:
dd_train.tail(3)

,Unnamed: 0,id,vas_id,buy_time,target
831650,831650,158236,2.0,1546203600,0.0
831651,831651,1825525,2.0,1546203600,0.0
831652,831652,3022610,2.0,1546203600,0.0


In [8]:
%%time
dd_test.head(3)

CPU times: user 25.1 ms, sys: 0 ns, total: 25.1 ms
Wall time: 69.5 ms


,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400


In [9]:
dd_test.tail(3)

,Unnamed: 0,id,vas_id,buy_time
71228,71228,1891350,2.0,1548018000
71229,71229,2437172,2.0,1548018000
71230,71230,988236,2.0,1548018000


#### Из данных видно что "Unnamed: 0" является индексом -> "index"
#### Так же для отличия колонок из датасета признаков и датасета train и test при обединении данных нужно переименовать колонки 
Для dd_features:

    "buy_time" -> "buy_time_fea"
    "index" -> "index_fea"

Для dd_train и dd_test :
    
    "buy_time" -> "buy_time_sub"
    "index" -> "index_sub"
(feature -> fea, subscriber -> sub)


In [10]:
%%time
dd_features = dd_features.rename(columns=get_replace_dict('fea'))
dd_train = dd_train.rename(columns=get_replace_dict('sub'))
dd_test = dd_test.rename(columns=get_replace_dict('sub'))

CPU times: user 75.7 ms, sys: 7.55 ms, total: 83.2 ms
Wall time: 82 ms


In [11]:
dd_train.head(3)

,index_sub,id,vas_id,buy_time_sub,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0


In [12]:
dd_test.tail(3)

,index_sub,id,vas_id,buy_time_sub
71228,71228,1891350,2.0,1548018000
71229,71229,2437172,2.0,1548018000
71230,71230,988236,2.0,1548018000


### Объединение данных для train и features

In [13]:
%%time
merged_data_train = dd.merge(dd_train, dd_features, on=['id'], how='inner')

CPU times: user 81 ms, sys: 551 µs, total: 81.5 ms
Wall time: 81.4 ms


In [14]:
%%time
merged_data_train.head(3)

CPU times: user 170 ms, sys: 23.2 ms, total: 193 ms
Wall time: 1.5 s


,index_sub,id,vas_id,buy_time_sub,target,index_fea,buy_time_fea,0,1,2,...,243,244,245,246,247,248,249,250,251,252
0,140,4130548,2.0,1544389200,0.0,8832,1548018000,11.700029,17.790888,4.429214,...,-943.373846,-598.770792,-25.996269,-22.630448,-220.747724,-14.832889,-0.694428,-12.175933,-0.45614,1.0
1,842,540997,1.0,1541365200,0.0,11897,1545598800,-96.799971,-69.199112,-108.200786,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-24.832889,0.305572,-12.175933,-0.45614,0.0
2,843,540997,4.0,1542574800,1.0,11897,1545598800,-96.799971,-69.199112,-108.200786,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-24.832889,0.305572,-12.175933,-0.45614,0.0


### Создание признака абсолютной разницы в метках предложения и отклика на предложениеоб услуге

In [15]:
%%time
merged_data_train['delta_buy_time'] = abs(merged_data_train.buy_time_fea - merged_data_train.buy_time_sub)
merged_data_train.head(10)

CPU times: user 361 ms, sys: 28.3 ms, total: 389 ms
Wall time: 2.56 s


,index_sub,id,vas_id,buy_time_sub,target,index_fea,buy_time_fea,0,1,2,...,244,245,246,247,248,249,250,251,252,delta_buy_time
0,140,4130548,2.0,1544389200,0.0,8832,1548018000,11.700029,17.790888,4.429214,...,-598.770792,-25.996269,-22.630448,-220.747724,-14.832889,-0.694428,-12.175933,-0.45614,1.0,3628800
1,842,540997,1.0,1541365200,0.0,11897,1545598800,-96.799971,-69.199112,-108.200786,...,-613.770792,-25.996269,-37.630448,-306.747724,-24.832889,0.305572,-12.175933,-0.45614,0.0,4233600
2,843,540997,4.0,1542574800,1.0,11897,1545598800,-96.799971,-69.199112,-108.200786,...,-613.770792,-25.996269,-37.630448,-306.747724,-24.832889,0.305572,-12.175933,-0.45614,0.0,3024000
3,926,563378,2.0,1545598800,0.0,12020,1534107600,-96.799971,-242.079112,-110.740786,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,11491200
4,1312,37,1.0,1534712400,0.0,9746,1545598800,-96.799971,-408.179112,-110.740786,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,10886400
5,1451,943725,1.0,1535317200,0.0,2204,1537736400,-74.179971,-356.919112,-88.120786,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,2419200
6,1586,361210,1.0,1538946000,0.0,1686,1537131600,-96.799971,-111.569112,-110.740786,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,1814400
7,1989,428569,1.0,1540155600,0.0,1771,1537131600,-94.679971,104.530888,-10.740786,...,-613.770792,-25.996269,-30.630448,-0.747724,15.167111,7.305572,20.824067,-0.45614,0.0,3024000
8,2003,518736,5.0,1537131600,0.0,11792,1544994000,100.950029,71.280888,87.009214,...,627.229208,9.003731,201.369552,295.252276,50.167111,-0.694428,-12.175933,-0.45614,0.0,7862400
9,2272,1387665,2.0,1544994000,0.0,4148,1544389200,64.220029,-233.919112,55.359214,...,-586.770792,-25.996269,-37.630448,-155.747724,-18.832889,-0.694428,-12.175933,-0.45614,0.0,604800


In [16]:
%%time
merged_data_train = merged_data_train.sort_values(['delta_buy_time'], inplace=True, ignore_index=True)
merged_data_train.head(3)

CPU times: user 1min 22s, sys: 8.24 s, total: 1min 30s
Wall time: 7min 57s


,index_sub,id,vas_id,buy_time_sub,target,index_fea,buy_time_fea,0,1,2,...,244,245,246,247,248,249,250,251,252,delta_buy_time
15,4477,3769599,5.0,1540760400,0.0,7368,1540760400,199.810029,-106.489112,185.869214,...,-613.770792,-25.996269,-37.630448,-300.747724,-25.832889,-0.694428,-12.175933,-0.45614,1.0,0
797,281030,391330,1.0,1534107600,0.0,3013727,1534107600,-96.799971,-310.959112,-110.740786,...,-612.770792,-25.996269,-37.630448,-292.747724,-16.832889,-0.694428,-3.175933,-0.45614,0.0,0
818,290873,1195974,5.0,1544389200,0.0,3016321,1544389200,-96.799971,-406.649112,-110.740786,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,0


#### Удаление дубликатов по "index_sub" с сохранением первого после сортировки

In [17]:
%%time
merged_data_train = merged_data_train.drop_duplicates(subset=['index_sub'], keep='first') 
merged_data_train.head(3)

KeyboardInterrupt: 

In [18]:
len(merged_data_train)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 